# Project: Exploring a Soccer Match Database

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

I have chosen to analyze the [European Soccer Database](https://www.kaggle.com/hugomathien/soccer). The database includes approximately 26,000 records of soccer matches, including team layout and game outcome. There is also data regarding player and team attributes (taken from the FIFA video games), and values for betting odds on each game across several online gambling platforms.

My focus will be on the players making up each team, team formations, and their respective impacts on team performance. Through this project, I will attempt to draw conclusions about factors influencing team performance based on the available data.

I chose this database and to focus on performance because of the connection with my work background. I am interested in improving the performance of a manufacturing organization which produces only custom made-to-order products. I would like to draw a parallel between analyzing team sport performance and team business performance.

### Importing

First I import the packages relevant for this project:

In [1]:
import sqlite3                      # to read the raw database file in .sqlite format
import pandas as pd                 # for creating and modifying dataframes
from matplotlib.pyplot import plot  # for data visualization
import seaborn as sb                # to clean up visualizations
import os                           # to locate files within the directory
import numpy as np                  #

# visualizations will render in-browser
%matplotlib inline                  

<a id='wrangling'></a>
## Data Wrangling

The file comes in .sqlite format, so it must be unpacked and imported into dataframes to be manipulated. Below, I will check out what data is included with each table and begin to shape the data so that it is useful for my analysis.

### General Properties

In [2]:
# Load your data and print out a few lines. Perform operations to inspect data
#   types and look for instances of missing or possibly errant data.


path = os.getcwd()                     # ensures that the full path is being used
database = path + '\\database.sqlite'  # even though the file should be in the same folder

con = sqlite3.connect(database)        # establish a connection with the database
tables = pd.read_sql(                  # write a query to see all tables
    """
    SELECT * FROM sqlite_master
    WHERE type='table';
    """,con=con)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


### Tables
I know from reading the documentation provided with the database that the 2 `_Attributes` tables are based on data from the FIFA video games. I will not need them to support my analysis, so they will not be brought into dataframes.

The 

In [3]:
# write query to import player table
player_df = pd.read_sql( 
    """
    SELECT player_api_id as id, player_name, birthday, height, weight 
    FROM Player;
    """, con=con, index_col='id', parse_dates=['birthday'])

# observe column names and datatypes
player_df.info()         

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11060 entries, 505942 to 39902
Data columns (total 4 columns):
player_name    11060 non-null object
birthday       11060 non-null datetime64[ns]
height         11060 non-null float64
weight         11060 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 432.0+ KB


In [4]:
# query to import league table
league_df = pd.read_sql(
    """
    SELECT * FROM League;
    """, con=con, index_col='id')

# observe column names and datatypes
league_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 1 to 24558
Data columns (total 2 columns):
country_id    11 non-null int64
name          11 non-null object
dtypes: int64(1), object(1)
memory usage: 264.0+ bytes


In [5]:
team_df = pd.read_sql(
    """
    SELECT * FROM team;
    """, con=con, index_col='team_api_id')
team_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 9987 to 7896
Data columns (total 4 columns):
id                  299 non-null int64
team_fifa_api_id    288 non-null float64
team_long_name      299 non-null object
team_short_name     299 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 11.7+ KB


Because the `Match` table contains the majority of the data pertinent to my analysis, I will try to `JOIN` the other tables as early as possible so that I can drop everything that I don't need and avoid doing any more operations.

First I pull the `Country` and `League` names directly in, using an `INNER JOIN` on their respective ID's.

I also have the `%%time` magic called because this table is so large; I want to also keep track of how long operations take and attempt to remove unnecessary steps at the end.

In [6]:
match_df = pd.read_sql(
    """
    SELECT Country.name as country, League.name as league, match.*
    FROM match
    JOIN Country ON Country.id = match.country_id
    JOIN League ON League.id = match.league_id;
    """, con=con, index_col='id', parse_dates=['date'])
match_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25979 entries, 1 to 25979
Columns: 116 entries, country to BSA
dtypes: datetime64[ns](1), float64(96), int64(8), object(11)
memory usage: 23.2+ MB


One of the first issues I will need to tackle is reducing the amount of columns in this table. The above output shows that there are 116 columns, too many to be listed using `.info()`. I will need to print a list of the columns to take a look.

In [7]:
for num, column in enumerate(list(match_df)):
    print('%i: %s' % (num,column))

0: country
1: league
2: country_id
3: league_id
4: season
5: stage
6: date
7: match_api_id
8: home_team_api_id
9: away_team_api_id
10: home_team_goal
11: away_team_goal
12: home_player_X1
13: home_player_X2
14: home_player_X3
15: home_player_X4
16: home_player_X5
17: home_player_X6
18: home_player_X7
19: home_player_X8
20: home_player_X9
21: home_player_X10
22: home_player_X11
23: away_player_X1
24: away_player_X2
25: away_player_X3
26: away_player_X4
27: away_player_X5
28: away_player_X6
29: away_player_X7
30: away_player_X8
31: away_player_X9
32: away_player_X10
33: away_player_X11
34: home_player_Y1
35: home_player_Y2
36: home_player_Y3
37: home_player_Y4
38: home_player_Y5
39: home_player_Y6
40: home_player_Y7
41: home_player_Y8
42: home_player_Y9
43: home_player_Y10
44: home_player_Y11
45: away_player_Y1
46: away_player_Y2
47: away_player_Y3
48: away_player_Y4
49: away_player_Y5
50: away_player_Y6
51: away_player_Y7
52: away_player_Y8
53: away_player_Y9
54: away_player_Y10
55: awa

### Data Cleaning

It looks like I will want to lose all of the columns after 77, because they contain metrics that I am not interested in for this analysis. In this case, I find it much easier to lose the columns within Pandas rather than SQL, because I want to use a numbered range.

In [8]:
match_df = match_df[match_df.columns[:78]]

Now I will convert the other column values from ID numbers to their respective string values. I still need to do this for the home and away team names, but it was already tackled in the SQL `JOIN` earlier for the 'Country' and 'League' columns. All I need to do is drop those ID columns as well.

In [9]:
values_dict = team_df['team_long_name'].to_dict()
match_df['home_team_api_id'] = match_df['home_team_api_id'].astype('int')
match_df['home_team_name'] = match_df['home_team_api_id'].replace(values_dict)
match_df['away_team_api_id'] = match_df['away_team_api_id'].astype('int')
match_df['away_team_name'] = match_df['away_team_api_id'].replace(values_dict)

In [10]:
match_df.drop(['country_id', 'league_id', 'home_team_api_id', 'away_team_api_id'], axis=1, inplace=True)

In [11]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25979 entries, 1 to 25979
Data columns (total 76 columns):
country            25979 non-null object
league             25979 non-null object
season             25979 non-null object
stage              25979 non-null int64
date               25979 non-null datetime64[ns]
match_api_id       25979 non-null int64
home_team_goal     25979 non-null int64
away_team_goal     25979 non-null int64
home_player_X1     24158 non-null float64
home_player_X2     24158 non-null float64
home_player_X3     24147 non-null float64
home_player_X4     24147 non-null float64
home_player_X5     24147 non-null float64
home_player_X6     24147 non-null float64
home_player_X7     24147 non-null float64
home_player_X8     24147 non-null float64
home_player_X9     24147 non-null float64
home_player_X10    24147 non-null float64
home_player_X11    24147 non-null float64
away_player_X1     24147 non-null float64
away_player_X2     24147 non-null float64
away_player_X

From looking at the remaining columns, I can see that many of the rows contain null values.

Below I will drop any rows containing null values using the `dropna` function.

In [12]:
match_df.dropna(how='any',inplace=True)

In [13]:
match_df.info()
#for col in player_cols:
#    print(match_df[np.isnan(match_df[col])])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21361 entries, 146 to 25979
Data columns (total 76 columns):
country            21361 non-null object
league             21361 non-null object
season             21361 non-null object
stage              21361 non-null int64
date               21361 non-null datetime64[ns]
match_api_id       21361 non-null int64
home_team_goal     21361 non-null int64
away_team_goal     21361 non-null int64
home_player_X1     21361 non-null float64
home_player_X2     21361 non-null float64
home_player_X3     21361 non-null float64
home_player_X4     21361 non-null float64
home_player_X5     21361 non-null float64
home_player_X6     21361 non-null float64
home_player_X7     21361 non-null float64
home_player_X8     21361 non-null float64
home_player_X9     21361 non-null float64
home_player_X10    21361 non-null float64
home_player_X11    21361 non-null float64
away_player_X1     21361 non-null float64
away_player_X2     21361 non-null float64
away_player

We can now see that each of the columns contains the same number of non-null entries, so there are no empty fields. There are still over 21,000 records even after being selective with the data, which should be plenty for this analysis.

Now I have gotten from the original 116 columns down to 76, which is still not quite easily readable. This is partially because both teams have 11 players, each of which has 3 dedicated columns in this table:
- `home/away_player_N` - the API ID of the player in that position
- `home/away_player_XN`- the 'X' coordinate position of the player on the field
- `home/away_player_YN`- the 'Y' coordinate position of the player on the field

To me, it makes more sense to condense these 3 fields for each player into a dictionary which specifices the `X,Y` coordinate set of the player's location:
```
{player: (x_coord, y_coord)}
```

To begin, I make a list of all of the player-related columns (they all have the word 'player' in the name).

In [14]:
# a list comprehension of all the column names with the word 'player'
player_cols = [col for col in match_df.columns if 'player' in col]

In [15]:
players = {}

for i in range(1,12):
    home_str = 'home_player_'
    away_str = 'away_player_'
    players[home_str+str(i)] = (home_str+'X'+str(i), home_str+'Y'+str(i))
    players[away_str+str(i)] = (away_str+'X'+str(i), away_str+'Y'+str(i))

In [16]:
values_dict = player_df['player_name'].to_dict()

def posit_dict(x, y, z):
    return dict([(x,(int(y),int(z)))])

for player in players.keys():
    df_name = player + '_coords'
    player_x = players[player][0]
    player_y = players[player][1]
    match_df[player] = match_df[player].astype('int')
    match_df[player] = match_df[player].replace(values_dict)
    match_df[df_name] = match_df.apply(lambda x: posit_dict(x[player], x[player_x], x[player_y]), axis=1)
    match_df.drop([player, player_x, player_y], axis=1, inplace=True)

In [17]:
for num, column in enumerate(list(match_df)):
    print('%i: %s' % (num,column))

0: country
1: league
2: season
3: stage
4: date
5: match_api_id
6: home_team_goal
7: away_team_goal
8: home_team_name
9: away_team_name
10: home_player_1_coords
11: away_player_1_coords
12: home_player_2_coords
13: away_player_2_coords
14: home_player_3_coords
15: away_player_3_coords
16: home_player_4_coords
17: away_player_4_coords
18: home_player_5_coords
19: away_player_5_coords
20: home_player_6_coords
21: away_player_6_coords
22: home_player_7_coords
23: away_player_7_coords
24: home_player_8_coords
25: away_player_8_coords
26: home_player_9_coords
27: away_player_9_coords
28: home_player_10_coords
29: away_player_10_coords
30: home_player_11_coords
31: away_player_11_coords


Now it seems that we have a manageable number of columns, and we can be confident that each row contains all of the relevant information.

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1: Which team formations are the most effective?

In [18]:
match_df['Score_Delt'] = match_df['home_team_goal']-match_df['away_team_goal']
bins = [np.NINF, -0.1, 0.1, np.Inf]
match_df['Outcome'] = pd.cut(match_df['Score_Delt'], bins, labels=['away','tie','home'], include_lowest=True)
match_df[['Score_Delt','Outcome']].describe()

,Score_Delt
count,21361.000000
mean,0.382239
std,1.792698
min,-9.000000
25%,-1.000000
50%,0.000000
75%,1.000000
max,10.000000


In [19]:
match_df

,country,league,season,stage,date,match_api_id,home_team_goal,away_team_goal,home_team_name,away_team_name,...,home_player_8_coords,away_player_8_coords,home_player_9_coords,away_player_9_coords,home_player_10_coords,away_player_10_coords,home_player_11_coords,away_player_11_coords,Score_Delt,Outcome
id,,,,,,,,,,,,,,,,,,,,,
146,Belgium,Belgium Jupiler League,2008/2009,24,2009-02-27,493017,2,1,KV Mechelen,KRC Genk,...,"{'Koen Persoons': (6, 7)}","{'Balazs Toth': (6, 7)}","{'Romeo van Dessel': (8, 7)}","{'Elyaniv Barda': (8, 7)}","{'Wouter Vrancken': (4, 10)}","{'Stein Huysegems': (4, 10)}","{'Giuseppe Rossini': (6, 10)}","{'Jelle Vossen': (6, 10)}",1,home
154,Belgium,Belgium Jupiler League,2008/2009,25,2009-03-08,493025,1,3,KSV Cercle Brugge,Club Brugge KV,...,"{'Tony Sergeant': (6, 7)}","{'Nabil Dirar': (6, 7)}","{'Oleg Iachtchouk': (8, 7)}","{'Ronald Vargas': (8, 7)}","{'Stijn De Smet': (4, 10)}","{'Wesley Sonck': (4, 10)}","{'Honour Gombami': (6, 10)}","{'Joseph Akpala': (6, 10)}",-2,away
156,Belgium,Belgium Jupiler League,2008/2009,25,2009-03-07,493027,2,0,RSC Anderlecht,SV Zulte-Waregem,...,"{'Guillaume Gillet': (6, 7)}","{'Franck Berrier': (6, 7)}","{'Mbark Boussoufa': (8, 7)}","{'Khaleem Hyland': (8, 7)}","{'Oleksandr Iakovenko': (4, 10)}","{'Ernest Webnje Nfor': (4, 10)}","{'Tom De Sutter': (6, 10)}","{'Thomas Matton': (6, 10)}",2,home
163,Belgium,Belgium Jupiler League,2008/2009,26,2009-03-13,493034,2,1,KV Mechelen,RSC Anderlecht,...,"{'Joachim Mununga': (6, 7)}","{'Mbark Boussoufa': (6, 7)}","{'Romeo van Dessel': (8, 7)}","{'Lucas Biglia': (8, 7)}","{'Bjoern Vleminckx': (4, 10)}","{'Guillaume Gillet': (4, 10)}","{'Giuseppe Rossini': (6, 10)}","{'Tom De Sutter': (6, 10)}",1,home
169,Belgium,Belgium Jupiler League,2008/2009,26,2009-03-14,493040,0,0,SV Zulte-Waregem,KSV Roeselare,...,"{'Franck Berrier': (6, 7)}","{'Vincent Provoost': (6, 7)}","{'Thomas Matton': (8, 7)}","{'Ivan Perisic': (8, 7)}","{'Ernest Webnje Nfor': (4, 10)}","{'Mahamadou Dissa': (4, 10)}","{'Khaleem Hyland': (6, 10)}","{'Sherjill MacDonald': (6, 10)}",0,tie
174,Belgium,Belgium Jupiler League,2008/2009,27,2009-03-22,493045,1,0,KAA Gent,SV Zulte-Waregem,...,"{'Milos Maric': (6, 7)}","{'Thomas Matton': (6, 7)}","{'Bryan Ruiz': (8, 7)}","{'Khaleem Hyland': (8, 7)}","{'Adnan Custovic': (4, 10)}","{'Ernest Webnje Nfor': (4, 10)}","{'Mbaye Leye': (6, 10)}","{'Chris Makiese': (6, 10)}",1,home
177,Belgium,Belgium Jupiler League,2008/2009,27,2009-03-21,493048,1,0,KSV Roeselare,KV Mechelen,...,"{'Ivan Perisic': (6, 7)}","{'Maxime Biset': (6, 7)}","{'Joerie Dequevy': (8, 7)}","{'Joachim Mununga': (8, 7)}","{'Mahamadou Dissa': (4, 10)}","{'Bjoern Vleminckx': (4, 10)}","{'Sherjill MacDonald': (6, 10)}","{'Giuseppe Rossini': (6, 10)}",1,home
190,Belgium,Belgium Jupiler League,2008/2009,29,2009-04-12,493061,1,0,RSC Anderlecht,Club Brugge KV,...,"{'Mbark Boussoufa': (6, 7)}","{'Nabil Dirar': (6, 7)}","{'Lucas Biglia': (8, 7)}","{'Vadis Odjidja-Ofoe': (8, 7)}","{'Guillaume Gillet': (4, 10)}","{'Wesley Sonck': (4, 10)}","{'Tom De Sutter': (6, 10)}","{'Joseph Akpala': (6, 10)}",1,home
191,Belgium,Belgium Jupiler League,2008/2009,29,2009-04-10,493062,1,0,KSV Roeselare,KRC Genk,...,"{'Ivan Perisic': (6, 7)}","{'Ederson Tormena': (6, 7)}","{'Joerie Dequevy': (8, 7)}","{'Stein Huysegems': (8, 7)}","{'Mahamadou Dissa': (4, 10)}","{'Tom de Mul': (4, 10)}","{'Sherjill MacDonald': (6, 10)}","{'Marvin Ogunjimi': (6, 10)}",1,home


In [20]:
home_coords_list = list(match_df.filter(regex='home.*_coords', axis=1))
away_coords_list = list(match_df.filter(regex='away.*_coords', axis=1))
home_temp_dict = match_df[home_coords_list].apply(lambda x: x.tolist(), axis=1)
away_temp_dict = match_df[away_coords_list].apply(lambda x: x.tolist(), axis=1)

In [21]:
home_formations = []
away_formations = []

for row in home_temp_dict:
    home_team_formation = []
    for dict in row:
        home_team_formation.append(list(dict.values())[0])
    home_formations.append(home_team_formation)
    
for row in away_temp_dict:
    away_team_formation = []
    for dict in row:
        away_team_formation.append(list(dict.values())[0])
    away_formations.append(away_team_formation)

home_formations = np.array(home_formations)
away_formations = np.array(away_formations)

In [22]:
print(home_formations.shape)
print(away_formations.shape)

(21361, 11, 2)
(21361, 11, 2)


In [23]:
match_df.tail()

,country,league,season,stage,date,match_api_id,home_team_goal,away_team_goal,home_team_name,away_team_name,...,home_player_8_coords,away_player_8_coords,home_player_9_coords,away_player_9_coords,home_player_10_coords,away_player_10_coords,home_player_11_coords,away_player_11_coords,Score_Delt,Outcome
id,,,,,,,,,,,,,,,,,,,,,
25973,Switzerland,Switzerland Super League,2015/2016,8,2015-09-13,1992089,3,3,FC Zürich,FC Thun,...,"{'Sangone Sarr': (6, 7)}","{'Gonzalo Zarate': (3, 8)}","{'Oliver Buff': (8, 7)}","{'Gianluca Frontino': (5, 8)}","{'Franck Etoundi': (4, 10)}","{'Nelson Ferreira': (7, 8)}","{'Davide Chiumiento': (6, 10)}","{'Roman Buess': (5, 11)}",0,tie
25975,Switzerland,Switzerland Super League,2015/2016,9,2015-09-22,1992091,1,0,FC St. Gallen,FC Thun,...,"{'Geoffrey Treand': (3, 8)}","{'Gonzalo Zarate': (3, 8)}","{'Danijel Aleksic': (5, 8)}","{'Gianluca Frontino': (5, 8)}","{'Yannis Tafer': (7, 8)}","{'Simone Rapp': (7, 8)}","{'Sandro Gotal': (5, 11)}","{'Roman Buess': (5, 11)}",1,home
25976,Switzerland,Switzerland Super League,2015/2016,9,2015-09-23,1992092,1,2,FC Vaduz,FC Luzern,...,"{'Joel Untersee': (8, 7)}","{'Remo Freuler': (7, 7)}","{'Markus Neumayr': (5, 9)}","{'Jahmir Hyka': (3, 10)}","{'Robin Kamber': (4, 11)}","{'Marco Schneuwly': (5, 10)}","{'Franz Burgmeier': (6, 11)}","{'Jakob Jantscher': (7, 10)}",-1,away
25977,Switzerland,Switzerland Super League,2015/2016,9,2015-09-23,1992093,2,0,Grasshopper Club Zürich,FC Sion,...,"{'Yoric Ravet': (3, 8)}","{'Ebenezer Assifuah': (3, 8)}","{'Shani Tarashaj': (5, 8)}","{'Carlitos': (5, 8)}","{'Caio': (7, 8)}","{'Edmilson Fernandes': (7, 8)}","{'Munas Dabbur': (5, 11)}","{'Moussa Konate': (5, 11)}",2,home
25979,Switzerland,Switzerland Super League,2015/2016,9,2015-09-23,1992095,4,3,BSC Young Boys,FC Basel,...,"{'Alain Rochat': (6, 7)}","{'Birkir Bjarnason': (3, 8)}","{'Miralem Sulejmani': (8, 7)}","{'Matias Emilio Delgado': (5, 8)}","{'Yuya Kubo': (4, 10)}","{'Shkelzen Gashi': (7, 8)}","{'Alexander Gerndt': (6, 10)}","{'Breel Embolo': (5, 11)}",1,home


### Research Question 2: Which players affect team performance the most?

In [24]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.

## Submitting your Project 

> Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!

In [25]:
#from subprocess import call
#call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])